<a href="https://colab.research.google.com/github/DivyaSadhana/Prediction-of-Hospitalization-rate-based-on-Age-Vaccine-Manufacturer-Number-of-days-for-event-Onse/blob/main/Decision_Tree_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
# Establish connection between My Drive and Google colab

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [43]:
# Import File

import pandas as pd
df = pd.read_excel(r'/content/gdrive/My Drive/2023_Master_Tracker.xlsx')
df

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,SYMPTOM3,SYMPTOM4,SYMPTOM5,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,2547730,2023-01-01,DE,53.00,NaN,NaN,F,NaN,The adverse event is that the patient went int...,NaN,...,Coma,Computerised tomogram,Exposure to SARS-CoV-2,COVID19,JANSSEN,1808982,UNK,SYR,AR,COVID19 (COVID19 (JANSSEN))
1,2547730,2023-01-01,DE,53.00,NaN,NaN,F,NaN,The adverse event is that the patient went int...,NaN,...,Laboratory test,Magnetic resonance imaging,SARS-CoV-2 antibody test negative,COVID19,JANSSEN,1808982,UNK,SYR,AR,COVID19 (COVID19 (JANSSEN))
2,2547730,2023-01-01,DE,53.00,NaN,NaN,F,NaN,The adverse event is that the patient went int...,NaN,...,X-ray,NaN,NaN,COVID19,JANSSEN,1808982,UNK,SYR,AR,COVID19 (COVID19 (JANSSEN))
3,2547731,2023-01-01,MA,6.00,6.0,NaN,M,NaN,Error: Incorrect Reconstitution-,NaN,...,NaN,NaN,NaN,COVID19,PFIZER\BIONTECH,GK1657,3,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
4,2547732,2023-01-01,MA,38.00,38.0,NaN,F,NaN,Error: Patient Accidentally Stuck by Needle-,NaN,...,NaN,NaN,NaN,COVID19-2,MODERNA,141H22A,4,IM,LA,COVID19 (COVID19 (MODERNA BIVALENT))
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155172,2717538,2023-11-24,CO,NaN,NaN,NaN,U,NaN,breast cancer; This spontaneous case was repor...,NaN,...,NaN,NaN,NaN,COVID19,MODERNA,NaN,UNK,NaN,NaN,COVID19 (COVID19 (MODERNA))
155173,2717539,2023-11-24,NY,71.00,NaN,NaN,F,NaN,I could barely lift my arm/ could barely move;...,NaN,...,Fatigue,Gait inability,Lymphadenitis,COVID19,MODERNA,NaN,6,NaN,NaN,COVID19 (COVID19 (MODERNA))
155174,2717539,2023-11-24,NY,71.00,NaN,NaN,F,NaN,I could barely lift my arm/ could barely move;...,NaN,...,Pain in extremity,NaN,NaN,COVID19,MODERNA,NaN,6,NaN,NaN,COVID19 (COVID19 (MODERNA))
155175,2624988,2023-05-02,WA,1.58,1.0,0.7,M,NaN,"Patient received monovalent vaccine on Friday,...",NaN,...,NaN,NaN,NaN,COVID19,MODERNA,AS1417B,1,IM,LL,COVID19 (COVID19 (MODERNA))


In [44]:
DataFrame = df[['AGE_YRS','NUMDAYS','VAX_MANU','HOSPITAL']]
DataFrame.head(2)

,AGE_YRS,NUMDAYS,VAX_MANU,HOSPITAL
0,53.0,NaN,JANSSEN,NaN
1,53.0,NaN,JANSSEN,NaN


In [45]:
from sklearn.metrics import confusion_matrix, f1_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.preprocessing import LabelEncoder

# Assuming DataFrame is your DataFrame
# Fill null values of all columns with 0
DataFrame = DataFrame.fillna(0)

# Ensure that all columns are of type string before performing label encoding
DataFrame = DataFrame.astype(str)

Features1 = DataFrame.drop('HOSPITAL', axis=1)  # Features (all columns except 'HOSPITAL')
Label1 = DataFrame['HOSPITAL']   # Target ('HOSPITAL' column)

# Exclude rows where 'NUMDAYS' > 31
Features1 = Features1[Features1['NUMDAYS'].astype(float) <= 31]
Label1 = Label1.loc[Features1.index]

# Split the data into training and testing sets
Features1_train, Features1_test, Label1_train, Label1_test = train_test_split(Features1, Label1, test_size=0.2, random_state=42, stratify=Label1)

# Print the number of training and test cases
print(f"Number of training cases: {len(Features1_train)}")
print(f"Number of test cases: {len(Features1_test)}")

# Transform 'HOSPITAL' rows value with "Y" to 1 and null values of 'HOSPITAL' to 0
Label1_train = Label1_train.apply(lambda x: 1 if x == 'Y' else 0)
Label1_test = Label1_test.apply(lambda x: 1 if x == 'Y' else 0)

# Transform 'VAX_MANU' performing LabelEncoder to convert categorical values of 'VAX_MANU' to integers
encoder = LabelEncoder()
Features1_train['VAX_MANU'] = encoder.fit_transform(Features1_train['VAX_MANU'])  # Already string, so no need to convert
Features1_test['VAX_MANU'] = encoder.transform(Features1_test['VAX_MANU'])  # Use transform() for test set to avoid data leakage

# Change data type of features, labels datasets of training, test dataset into integers
Features1_train = Features1_train.astype(float).astype(int)
Features1_test = Features1_test.astype(float).astype(int)
Label1_train = Label1_train.astype(int)
Label1_test = Label1_test.astype(int)

# Create a decision tree classifier and fit it to the training data
DecisionTree_Model = DecisionTreeClassifier(criterion='gini')
DecisionTree_Model.fit(Features1_train, Label1_train)

# Make predictions on the testing set
Label1_pred = DecisionTree_Model.predict(Features1_test)

# Calculate the accuracy and precision of the model
Accuracy = DecisionTree_Model.score(Features1_test, Label1_test)
Precision = precision_score(Label1_test, Label1_pred, average='weighted')  # Use 'weighted' for multi-class classification
F1 = f1_score(Label1_test, Label1_pred, average='weighted')  # Use 'weighted' for multi-class classification
Recall = recall_score(Label1_test, Label1_pred, average='weighted')  # Use 'weighted' for multi-class classification

# Calculate and print the confusion matrix
Confusion_Matrix = confusion_matrix(Label1_test, Label1_pred)

print(f"Accuracy: {Accuracy}")
print(f"Precision: {Precision}")
print(f"F1 Score: {F1}")
print(f"Recall: {Recall}")
print(f"Confusion Matrix: \n{Confusion_Matrix}")


Number of training cases: 85979
Number of test cases: 21495
Accuracy: 0.9377994882530821
Precision: 0.928253886479012
F1 Score: 0.9285579434154838
Recall: 0.9377994882530821
Confusion Matrix: 
[[19557   266]
 [ 1071   601]]


In [46]:
import pandas as pd

# Assuming your features are named 'Feature1', 'Feature2', 'Feature3'
Features1 = ['AGE_YRS','NUMDAYS','VAX_MANU']

# Your input data
data_input = [90, 22, 9]

# Convert your input data into a DataFrame with feature names
data_input_df = pd.DataFrame([data_input], columns=Features1)

# Now you can predict with your model
prediction = DecisionTree_Model.predict(data_input_df)

print(f"Input_Class: {data_input_df}")
print(f"Prediction_Class: {prediction}")

Input_Class:    AGE_YRS  NUMDAYS  VAX_MANU
0       90       22         9
Prediction_Class: [1]


In [47]:
import pandas as pd

# Assuming your features are named 'Feature1', 'Feature2', 'Feature3'
Features1 = ['AGE_YRS','NUMDAYS','VAX_MANU']

# Your input data
data_input = [90, 22, 8]

# Convert your input data into a DataFrame with feature names
data_input_df = pd.DataFrame([data_input], columns=Features1)

# Now you can predict with your model
prediction = DecisionTree_Model.predict(data_input_df)

print(f"Input_Class: {data_input_df}")
print(f"Prediction_Class: {prediction}")

Input_Class:    AGE_YRS  NUMDAYS  VAX_MANU
0       90       22         8
Prediction_Class: [0]


#Model 2

In [48]:
DataTable = df[['AGE_YRS','NUMDAYS','VAX_MANU','HOSPITAL']]
DataTable.head(2)

,AGE_YRS,NUMDAYS,VAX_MANU,HOSPITAL
0,53.0,NaN,JANSSEN,NaN
1,53.0,NaN,JANSSEN,NaN


In [50]:
from sklearn.metrics import confusion_matrix, f1_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Assuming DataTable is your DataFrame
# Fill null values of all columns with 0
DataTable = DataTable.fillna(0)

# Ensure that all columns are of type string before performing label encoding
DataTable = DataTable.astype(str)

Features1 = DataTable[['AGE_YRS', 'NUMDAYS', 'VAX_MANU']]  # Features ('AGE_YRS', 'NUMDAYS', 'VAX_MANU')
Label1 = DataTable['HOSPITAL']   # Target ('HOSPITAL' column)

# Split the data into training and testing sets
Features_train, Features_test, Label_train, Label_test = train_test_split(Features1, Label1, test_size=0.2, random_state=42, stratify=Label1)

# Transform 'HOSPITAL' rows value with "Y" to 1 and null values of 'HOSPITAL' to 0
Label_train = Label_train.apply(lambda x: 1 if x == 'Y' else 0)
Label_test = Label_test.apply(lambda x: 1 if x == 'Y' else 0)

# Exclude rows where 'NUMDAYS' > 31 in training and testing sets
Features_train = Features_train[Features_train['NUMDAYS'].astype(float) <= 31]
Label_train = Label_train.loc[Features_train.index]
Features_test = Features_test[Features_test['NUMDAYS'].astype(float) <= 31]
Label_test = Label_test.loc[Features_test.index]

# Exclude null values of 'AGE' and 'NUMDAYS' in training and testing sets
Features_train = Features_train[Features_train['AGE_YRS'].notnull() & Features_train['NUMDAYS'].notnull()]
Label_train = Label_train.loc[Features_train.index]
Features_test = Features_test[Features_test['AGE_YRS'].notnull() & Features_test['NUMDAYS'].notnull()]
Label_test = Label_test.loc[Features_test.index]

# Define the bin edges for 'AGE_YRS'
bins_age = [0, 30, 60, 90, float('inf')]
labels_age = [0, 1, 2, 3]

# Create a new column 'AGE_YRS_BINNED' by cutting the 'AGE_YRS' column into bins
Features_train['AGE_YRS_BINNED'] = pd.cut(Features_train['AGE_YRS'].astype(float), bins=bins_age, labels=labels_age, include_lowest=True)
Features_test['AGE_YRS_BINNED'] = pd.cut(Features_test['AGE_YRS'].astype(float), bins=bins_age, labels=labels_age, include_lowest=True)

# Define the bin edges for 'NUMDAYS'
bins_numdays = [0, 10, 20, 30, float('inf')]
labels_numdays = [0, 1, 2, 3]

# Create a new column 'NUMDAYS_BINNED' by cutting the 'NUMDAYS' column into bins
Features_train['NUMDAYS_BINNED'] = pd.cut(Features_train['NUMDAYS'].astype(float), bins=bins_numdays, labels=labels_numdays, include_lowest=True)
Features_test['NUMDAYS_BINNED'] = pd.cut(Features_test['NUMDAYS'].astype(float), bins=bins_numdays, labels=labels_numdays, include_lowest=True)

# Transform 'VAX_MANU' performing LabelEncoder to convert categorical values of 'VAX_MANU' to integers
encoder = LabelEncoder()
Features_train['VAX_MANU'] = encoder.fit_transform(Features_train['VAX_MANU'])  # Already string, so no need to convert
Features_test['VAX_MANU'] = encoder.transform(Features_test['VAX_MANU'])  # Use transform() for test set to avoid data leakage

# Select only the binned features and the encoded 'VAX_MANU'
Features_train = Features_train[['AGE_YRS_BINNED', 'NUMDAYS_BINNED', 'VAX_MANU']]
Features_test = Features_test[['AGE_YRS_BINNED', 'NUMDAYS_BINNED', 'VAX_MANU']]

# Change data type of features, labels datasets of training, test dataset into integers
Features_train = Features_train.astype(float).astype(int)
Features_test = Features_test.astype(float).astype(int)
Label_train = Label_train.astype(int)
Label_test = Label_test.astype(int)

# Create a decision tree classifier and fit it to the training data
DecisionTree_Model = DecisionTreeClassifier(criterion='gini')
DecisionTree_Model.fit(Features_train, Label_train)

# Make predictions on the testing set
Label_pred = DecisionTree_Model.predict(Features_test)

# Calculate the accuracy and precision of the model
Accuracy = DecisionTree_Model.score(Features_test, Label_test)
Precision = precision_score(Label_test, Label_pred, average='weighted')  # Use 'weighted' for multi-class classification
F1 = f1_score(Label_test, Label_pred, average='weighted')  # Use 'weighted' for multi-class classification
Recall = recall_score(Label_test, Label_pred, average='weighted')  # Use 'weighted' for multi-class classification

# Calculate and print the confusion matrix
Confusion_Matrix = confusion_matrix(Label_test, Label_pred)

print(f"Accuracy: {Accuracy}")
print(f"Precision: {Precision}")
print(f"F1 Score: {F1}")
print(f"Recall: {Recall}")
print(f"Confusion Matrix: \n{Confusion_Matrix}")

# Convert the features and labels to values
Features_train, Label_train = Features_train.values, Label_train.values
Features_test, Label_test = Features_test.values, Label_test.values

# Print the features and label for the first 5 patients
for n in range(0,5):
    print("Patient", str(n+1), "\n  Features:", list(Features_train[n]), "\n  Label:", Label_train[n])


Accuracy: 0.9245212691608815
Precision: 0.8963851313002474
F1 Score: 0.8974900972669971
Recall: 0.9245212691608815
Confusion Matrix: 
[[19729   117]
 [ 1503   114]]
Patient 1 
  Features: [2, 2, 11] 
  Label: 1
Patient 2 
  Features: [0, 0, 11] 
  Label: 0
Patient 3 
  Features: [2, 0, 11] 
  Label: 1
Patient 4 
  Features: [0, 0, 16] 
  Label: 0
Patient 5 
  Features: [0, 0, 22] 
  Label: 0


In [51]:
import pandas as pd

# Assuming your features are named 'Feature1', 'Feature2', 'Feature3'
Features1 = ['AGE_YRS_BINNED','NUMDAYS_BINNED','VAX_MANU']

# Your input data
data_input2 = [3, 2, 8]

# Convert your input data into a DataFrame with feature names
data_input_df2 = pd.DataFrame([data_input2], columns=Features1)

# Now you can predict with your model
prediction = DecisionTree_Model.predict(data_input_df2)

print(f"Input_Class: {data_input_df2}")
print(f"Prediction_Class: {prediction}")

Input_Class:    AGE_YRS_BINNED  NUMDAYS_BINNED  VAX_MANU
0               3               2         8
Prediction_Class: [1]


# Model 1 was better over Model 2 in predicting True Positives over false positives